In [53]:
from transformers.integrations import TensorBoardCallback
from torch.utils.tensorboard import SummaryWriter
from transformers import TrainingArguments
from transformers import Trainer, HfArgumentParser
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType
from dataclasses import dataclass, field
import datasets
import os
os.environ["WANDB_DISABLED"] = "False"

# 模型导入

In [2]:
model_dir = "/root/autodl-tmp/model/"
model_name = "chatglm"
model_path = os.path.join(model_dir,model_name)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, load_in_8bit=False, trust_remote_code=True, device_map="auto")
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.is_parallelizable = True
model.model_parallel = True
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [3]:
print(model)

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in_

# 模型训练

In [54]:
@dataclass
class CastOutputToFloat(nn.Sequential):
    def forward(self, x):
        return super().forward(x).to(torch.float32)

@dataclass
class FinetuneArguments:
    dataset_path: str = field(default="data/alpaca")
    model_path: str = field(default="output")
    lora_rank: int = field(default=8)

In [64]:
training_args = TrainingArguments(
    output_dir="/root/autodl-tmp/model/chatglm-lora",          # 输出目录
    evaluation_strategy="epoch",     # 评估策略
    per_device_train_batch_size=16,   # 每个设备的训练批量大小
    per_device_eval_batch_size=16,    # 每个设备的评估批量大小
    num_train_epochs=3,              # 训练的总轮数
    learning_rate=2e-5,              # 学习率
    logging_dir="./logs",            # 日志保存目录
    logging_steps=100,                # 每10步记录一次日志
    save_steps=500,                  # 每500步保存一次模型
    save_total_limit=2,              # 仅保留最近的两个检查点
)
finetune_args = FinetuneArguments(
    dataset_path ="/root/autodl-tmp/data/wenlv/wenlv_token",
    model_path ="/root/autodl-tmp/model/chatglm-lora",
    lora_rank = 16,
)

/root/miniconda3/lib/python3.8/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [65]:
writer = SummaryWriter()
print(finetune_args, training_args)

FinetuneArguments(dataset_path='/root/autodl-tmp/data/wenlv/wenlv_token', model_path='/root/autodl-tmp/model/chatglm-lora', lora_rank=16) TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_steps=

In [66]:
# class ModifiedTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         return model(input_ids=inputs["input_ids"],labels=inputs["labels"],).loss
#     def save_model(self, output_dir=None, _internal_call=False):
#         from transformers.trainer import TRAINING_ARGS_NAME
#         os.makedirs(output_dir, exist_ok=True)
#         torch.save(self.args, os.path.join(output_dir, TRAINING_ARGS_NAME))
#         saved_params = {
#             k: v.to("cpu") for k, v in self.model.named_parameters() if v.requires_grad
#         }
#         torch.save(saved_params, os.path.join(output_dir, "lora_model.bin"))

In [67]:
peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=finetune_args.lora_rank,
        lora_alpha=32,
        lora_dropout=0.1,
)
#model打上了lora的外挂了，会把原有参数都冻结住
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 3,899,392 || all params: 6,247,483,392 || trainable%: 0.0624


In [68]:
train_dataset = datasets.load_from_disk(finetune_args.dataset_path)
print(f"\n{len(train_dataset)=}\n")
index=1650
example = train_dataset[1000]
print(example)
print(tokenizer.decode(example['input_ids'], skip_special_tokens=True))


len(train_dataset)=2956

{'input_ids': [64790, 64792, 29101, 30954, 30910, 51074, 55177, 34953, 30970, 30938, 33123, 13, 4244, 1902, 266, 30954, 30910, 48576, 57504, 36841, 37495, 54638, 2], 'seq_len': 17}
Instruction: 景德镇有哪些5A景区
Answer:  古窑民俗博览区


In [69]:
def collate_data(features):
    # print(features)
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids)
    input_ids = []
    labels_list = []
    index=0
    for ids_l, feature in sorted(zip(len_ids, features), key=lambda x: -x[0]):
        seq_len = feature['input_ids'].index(13)+1
        # print(feature['input_ids'])
        # print(feature['input_ids'][0:seq_len])
        # print(tokenizer.decode(feature['input_ids'][0:seq_len], skip_special_tokens=True))
        ids = feature["input_ids"]
        labels = ([-100] * (seq_len - 1) + ids[(seq_len - 1) :] + [-100] * (longest - ids_l))
        #print(labels)
        ids = ids + [tokenizer.pad_token_id] * (longest - ids_l)
        _ids = torch.LongTensor(ids)
        labels_list.append(torch.LongTensor(labels))
        input_ids.append(_ids)
        # print("input_ids",tokenizer.decode(input_ids[index].detach().tolist(), skip_special_tokens=True))
        # print("labels_list",tokenizer.decode(labels_list[index].detach().tolist(), skip_special_tokens=True))
        index+=1
    input_ids = torch.stack(input_ids)
    labels = torch.stack(labels_list)
    return { "input_ids": input_ids,"labels": labels,}

# trainer = ModifiedTrainer(model=model,train_dataset=train_dataset,eval_dataset=train_dataset,args=training_args,data_collator=collate_data,)#
trainer = Trainer(model=model,train_dataset=train_dataset,eval_dataset=train_dataset,args=training_args,data_collator=collate_data,)#
trainer.train()
writer.close()
# save model
model.save_pretrained(training_args.output_dir)

Epoch,Training Loss,Validation Loss
1,2.735700,2.103516
2,2.080900,2.025391
3,2.083600,2.015625


# 模型测试

In [1]:
from transformers import AutoTokenizer, AutoModel
from peft import PeftModel
from peft import PrefixEncoder, PrefixTuningConfig

In [3]:
#分词器 仍然用原生的
tokenizer = AutoTokenizer.from_pretrained("/root/autodl-tmp/model/chatglm", trust_remote_code=True)
#加载训练好的模型模型
model = AutoModel.from_pretrained("/root/autodl-tmp/model/chatglm", trust_remote_code=True).cuda()
model_lora = PeftModel.from_pretrained(model, "/root/autodl-tmp/model/chatglm-lora").half()

In [9]:
input="江西婺源攻略旅游路线"#"组织大学部门六七个人去武功山旅游，求攻略，路线"
response, history = model.chat(tokenizer, input, history=[])
print(response)
print("______________________________________________")
response, history = model_lora.chat(tokenizer, input, history=[])
print(response)

江西婺源是一处以自然风光和人文景观著称的旅游胜地,以下是一些推荐的旅游路线:

第一天:
1. 早上:从南昌出发,乘坐火车或长途汽车前往婺源。
2. 上午:游览婺源最著名的景点——河岭村。该村庄以独特的民俗文化和建筑风格而闻名,是一个适合漫步和拍照的地方。
3. 中午:在河岭村附近的餐厅享用当地美食。
4. 下午:参观江湾景区。这里有美丽的湖泊和山峦,可以欣赏到壮丽的景色。
5. 晚上:返回南昌。

第二天:
1. 早上:在婺源县城享用早餐。
2. 上午:游览紫清山。这是一座海拔较高的山,可以欣赏到壮丽的山峰和云海。
3. 中午:在紫清山的餐馆享用午餐。
4. 下午:参观江湾景区。在这里,可以欣赏到美丽的湖泊和山峦,可以在这里漫步或骑行。
5. 晚上:在婺源县城享用晚餐。

第三天:
1. 早上:在婺源县城享用早餐。
2. 上午:前往石城山。这是一座海拔较高的山,可以欣赏到壮丽的山峰和云海。
3. 中午:在石城山的餐馆享用午餐。
4. 下午:游览清华园。这是一个美丽的园林,有着独特的建筑风格和文化底蕴。
5. 晚上:在婺源县城享用晚餐。

第四天:
1. 早上:在婺源县城享用早餐。
2. 上午:前往三清山。这是一座海拔较高的山,有着独特的自然风光和文化底蕴。
3. 中午:在三清山的餐馆享用午餐。
4. 下午:游览河岭村。这是一个有着独特民俗文化和建筑风格的村庄,非常适合漫步和拍照。
5. 晚上:返回南昌。

以上是一些推荐的江西婺源旅游路线,当然还有其他很多值得游览的景点,可以根据自己的兴趣和时间进行安排。
______________________________________________
江西婺源是一处以自然风光和人文景观著称的旅游胜地,下面是一份较为详细的婺源旅游路线:

第一天:
1. 从南昌出发,乘坐火车或长途汽车前往婺源。
2. 抵达婺源后,前往江湾景区,游览长约3公里的小桥流水、粉墙黛瓦的徽派建筑群落,欣赏美丽的水乡风光。
3. 中午在景区内的餐厅品尝当地特色美食,如板栗、糊豆腐、糯米饭团等。
4. 下午前往江湾古镇,游览保存完好的明清建筑群,体验古镇的风情古韵,品尝当地小吃。
5. 晚上返回酒店休息。

第二天:
1. 早餐后前往篁岭景区,游览拥有独特地形的紫微山,登顶远眺,俯瞰整个景区的美景。
2. 中午在景区内的餐厅品尝当地特色美食,如香